# **Microsoft Defender Threat Intelligence**

### Jupyter NoteBook using the MDTI to build a simple Heatmaps

#### Author: 
Dennis Mercer  
Sr Program Manager  
Microsoft CxE MDTI  

## MDTI Heatmap Jupyter Notebook

MDTI Heatmap Jupyter Notebook is a Python script that utilizes the Microsoft Graph Security API to retrieve and analyze passive DNS information for a given domain. This notebook generates a heatmap visualization to display the first and last seen dates of various DNS record types (NS, SOA, AAAA) associated with the specified domain.

### Prerequisites

Before running the notebook, ensure you have the following:

- Python 3.x installed on your system.
- The required Python packages installed: `azure-identity`, `msgraph-sdk-security`, `pandas`, `matplotlib`, and `seaborn`.

### Setup

1. Clone the repository or download the notebook file to your local machine.
2. Install the required Python packages by running the following command in your terminal or command prompt:

```shell
pip install azure-identity msgraph-sdk-security pandas matplotlib seaborn




In [ ]:
pip install azure-identity msgraph-sdk-security pandas matplotlib seaborn

______________________________________________________________________________________________________________________________________________
#### The next cell had all of the potential import statement needed for all cells

In [ ]:
import os
import asyncio
from azure.identity.aio import ClientSecretCredential
from kiota_authentication_azure.azure_identity_authentication_provider import AzureIdentityAuthenticationProvider
from msgraph import GraphRequestAdapter
from msgraph import GraphServiceClient
from IPython.display import display
import ipywidgets as widgets
from azure.identity import ClientSecretCredential
import requests
import pandas as pd
import datetime
from datetime import timedelta
import matplotlib.pyplot as plt
import seaborn as sns
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.plotting import figure, show
from bokeh.io import output_notebook


In [ ]:
# Get the client secret from a local file
credential = ClientSecretCredential(
    tenant_id='Enter Your Azure Tenant ID',     # Your Azure AD tenant ID
    client_id='Enter Your App Client ID',       # Your Application Client ID
    tenant_id='Enter your Azure Tenant ID',     # Your Azure AD tenant ID
    client_id='Enter your App Client ID',       # Your Application Client ID
    client_secret = 'Enter your Client Secret'  # Your application client Secret which should never be stored in plain text
)
scopes = ['https://graph.microsoft.com/.default']  # Scopes or permissions required for API access

async def get_pdns(hostname: str):
    auth_provider = AzureIdentityAuthenticationProvider(credential, scopes=scopes)
    request_adapter = GraphRequestAdapter(auth_provider)

    client = GraphServiceClient(request_adapter)
    
    passive_dns = client.security.threat_intelligence.hosts.by_host_id(hostname).passive_dns
    my_request_configuration = passive_dns.PassiveDnsRequestBuilderGetRequestConfiguration()
    my_request_parameters =  passive_dns.PassiveDnsRequestBuilderGetQueryParameters()
    my_request_parameters.filter = "recordType in ('NS', 'SOA', 'AAAA')"
    my_request_configuration.query_parameters = my_request_parameters
    pdns = await passive_dns.get(request_configuration=my_request_configuration)
    rows = []
    if pdns:
       for pdns_record in pdns.value:
           first_seen_date = pdns_record.first_seen_date_time.date() if pdns_record.first_seen_date_time else None
           last_seen_date = pdns_record.last_seen_date_time.date() if pdns_record.last_seen_date_time else None
           rows.append([first_seen_date, last_seen_date])
    return rows

# Prompt the user for a domain
domain_input = input("Enter a domain to check: ")

print("Domain to check:", domain_input)  # Print the domain

table_rows = await get_pdns(domain_input)

df = pd.DataFrame(table_rows, columns=['First Seen', 'Last Seen'])

# Create a pivot table with the count of occurrences
pivot_table = df.groupby(['First Seen', 'Last Seen']).size().unstack(fill_value=0)

# Create the heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(pivot_table, cmap='Blues', annot=True, fmt='g')
#plt.title('Occurrences Heatmap')
plt.title('Entity: ' + domain_input)
plt.xlabel('Last Seen')
plt.ylabel('First Seen')
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.show()

________________________________________________________________________________________________________________________________________________________
#### Another method for heatmap, which produces a Bar Chart of resolutions going back 18 months.

In [ ]:
# Get the client secret from a local file
credential = ClientSecretCredential(
    tenant_id='Enter your Azure Tenant ID',     # Your Azure AD tenant ID
    client_id='Enter your App Client ID',       # Your Application Client ID
    client_secret = 'Enter your Client Secret'  # Your application client Secret which should never be stored in plain text
)

scopes = ['https://graph.microsoft.com/.default']  # Scopes or permissions required for API access

async def get_pdns(hostname: str):
    auth_provider = AzureIdentityAuthenticationProvider(credential, scopes=scopes)
    request_adapter = GraphRequestAdapter(auth_provider)

    client = GraphServiceClient(request_adapter)

    passive_dns = client.security.threat_intelligence.hosts.by_host_id(hostname).passive_dns
    my_request_configuration = passive_dns.PassiveDnsRequestBuilderGetRequestConfiguration()
    my_request_parameters = passive_dns.PassiveDnsRequestBuilderGetQueryParameters()
    my_request_parameters.filter = "recordType in ('A', 'AAAA')"
    my_request_configuration.query_parameters = my_request_parameters
    pdns = await passive_dns.get(request_configuration=my_request_configuration)
    
    records = []
    if pdns:
        for x in pdns.value:
            record = {
                'Artifact ID': x.artifact.id,
                'Record Type': x.record_type,
                'First Seen': x.first_seen_date_time.date().strftime('%m/%d/%Y') if x.first_seen_date_time else None,
                'Last Seen': x.last_seen_date_time.date().strftime('%m/%d/%Y') if x.last_seen_date_time else None
            }
            records.append(record)
    
    return records

# Prompt the user for a domain
domain = input("Enter a domain to check: ")

# Fetch the passive DNS records for the domain
results = await get_pdns(domain)

# Create a DataFrame with the results
df = pd.DataFrame(results)

# Create a new DataFrame with start and end columns
df_gantt = df.copy()

# Drop rows with missing start or end dates
df_gantt.dropna(subset=['First Seen', 'Last Seen'], inplace=True)

# Convert First Seen and Last Seen columns to datetime
df_gantt['First Seen'] = pd.to_datetime(df_gantt['First Seen'])
df_gantt['Last Seen'] = pd.to_datetime(df_gantt['Last Seen'])

# Calculate the duration for each record
df_gantt['Duration'] = df_gantt['Last Seen'] - df_gantt['First Seen']

# Filter records from the last 18 months
eighteen_months_ago = datetime.now() - timedelta(days=18*30)
df_gantt = df_gantt[df_gantt['First Seen'] >= eighteen_months_ago]

# Sort the DataFrame by the First Seen column
df_gantt.sort_values('First Seen', inplace=True)

# Reset the index
df_gantt.reset_index(drop=True, inplace=True)

output_notebook()

source = ColumnDataSource(df_gantt)

TOOLTIPS = [
    ("First Seen", "@{First Seen}{%F}"),
    ("Last Seen", "@{Last Seen}{%F}")
]

p = figure(y_range=df_gantt['Artifact ID'].unique(), x_axis_type='datetime', title=f"Heatmap: {domain}",
           height=600, width=800, tooltips=TOOLTIPS)

p.hbar(y='Artifact ID', left='First Seen', right='Last Seen', height=0.4, source=source)

p.yaxis.axis_label = 'IP ADDRESS'
p.xaxis.axis_label = 'Timeline'

# Add formatters for the tooltip values
p.hover.formatters = {'@{First Seen}': 'datetime', '@{Last Seen}': 'datetime'}

show(p)
